In [2]:
pip install pandas mysql-connector-python

  Obtaining dependency information for mysql-connector-python from https://files.pythonhosted.org/packages/d9/91/007a0d60fee8db4f7385075dc50bf62d2d359b417b374ec06b06ce6c2d64/mysql_connector_python-8.3.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/15.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.4 MB ? eta -:--:--
   ---------------------------------------- 0.1/15.4 MB 656.4 kB/s eta 0:00:24
    --------------------------------------- 0.3/15.4 MB 2.1 MB/s eta 0:00:08
   - -------------------------------------- 0.7/15.4 MB 4.3 MB/s eta 0:00:04
   ---- ----------------------------------- 1.7/15.4 MB 7.9 MB/s eta 0:00:02
   -------- ------------------------------- 3.1/15.4 MB 11.6 MB/s eta 0:00:02
   ----------- ---------------------------- 4.5/15.4 MB 14.2 MB/s eta 0:00:01
   -------------- ------------------------- 5.6/15.4 MB 15.7 MB/s eta 0:00:01
   -----

In [34]:
import pandas as pd
import mysql.connector

config = {
    'user': 'root',
    'password': 'vanillefootie123',
    'host': 'localhost',  
    'database': 'sakila',
}

In [35]:
conn = mysql.connector.connect(**config)

In [36]:
rentalsmay = """select customer_id, count(*)
                    from rentals_may
                        group by customer_id 
                            order by count(*) desc;"""

In [37]:
rentalsjune = """select customer_id, count(*)
                    from rentals_june
                        group by customer_id
                            order by count(*) desc;"""

In [38]:
df1 = pd.read_sql_query(rentalsmay, conn)
df1

C:\Users\torei\AppData\Local\Temp\ipykernel_15884\3349291050.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql_query(rentalsmay, conn)


,customer_id,count(*)
0,197,8
1,506,7
2,109,7
3,269,6
4,239,6
...,...,...
515,431,1
516,351,1
517,10,1
518,136,1


In [39]:
df2 = pd.read_sql_query(rentalsjune, conn)
df2

C:\Users\torei\AppData\Local\Temp\ipykernel_15884\3727820648.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql_query(rentalsjune, conn)


,customer_id,count(*)
0,31,11
1,454,10
2,329,9
3,295,9
4,561,9
...,...,...
585,425,1
586,412,1
587,335,1
588,22,1


In [40]:
def compare_rentals_may_june(df1, df2):
    merged_df = pd.merge(df1, df2, on='customer_id', how='outer', suffixes=('_may', '_june'))
    
    merged_df.fillna(0, inplace=True)
    
    merged_df['comparison'] = merged_df['count(*)_june'] - merged_df['count(*)_may']
    
    merged_df['comparison_result'] = merged_df['comparison'].apply(lambda x: 'More' if x > 0 else 'Less' if x < 0 else 'Equal')
    
    return merged_df

In [41]:
comparison_result = compare_rentals_may_june(df1, df2)
comparison_result

,customer_id,count(*)_may,count(*)_june,comparison,comparison_result
0,197,8.0,8.0,0.0,Equal
1,506,7.0,5.0,-2.0,Less
2,109,7.0,5.0,-2.0,Less
3,269,6.0,3.0,-3.0,Less
4,239,6.0,5.0,-1.0,Less
...,...,...,...,...,...
593,63,0.0,1.0,1.0,More
594,598,0.0,1.0,1.0,More
595,487,0.0,1.0,1.0,More
596,555,0.0,1.0,1.0,More
